In [20]:
import os
import pandas as pd
import numpy as np
import sklearn.manifold
import matplotlib.pyplot as plt
import nltk
import regex as re
import re
import codecs
import csv
import glob
import multiprocessing
from nltk.corpus import stopwords
from nltk.stem.snowball import SnowballStemmer
from nltk.tokenize import word_tokenize, sent_tokenize
stemmer = SnowballStemmer("english")
import math
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.cluster import KMeans
import seaborn as sns
import sklearn

In [21]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import confusion_matrix
from sklearn.metrics import f1_score
from sklearn.metrics import accuracy_score
from sklearn.feature_extraction.text import TfidfVectorizer
from nltk.corpus import stopwords
from nltk.stem.wordnet import WordNetLemmatizer
import string
import re
from collections import Counter
 
stop = set(stopwords.words('english'))
exclude = set(string.punctuation)
lemma = WordNetLemmatizer()

In [32]:
df = pd.read_csv("C:\\Users\\alvin\\Desktop\\all_good_articles1.csv")
labels_df = pd.read_csv("C:\\Users\\alvin\\Desktop\\Labeled Articles.csv")

In [33]:
df.head()

,Unnamed: 0,index,Unnamed: 0.1,id,title,publication,author,date,year,month,url,content
0,45379,105029,108489,162183,How Nirvana’s ’Smells Like Teen Spirit’ Became...,NPR,Guy Raz,2000-05-15,2000.0,5.0,http://www.npr.org/2000/05/15/1074173/smells-l...,"In the early 1990s, Seattle stood at the cente..."
1,4397,62169,65490,89568,Everything changes. Everything. Especially,Talking Points Memo,NaN,2003-06-14,2003.0,6.0,https://web.archive.org/web/20170222000427/htt...,Part of America’s special nature good and ...
2,48653,108303,111764,166319,David Bowie On The Ziggy Stardust Years: ’We W...,NPR,NPR Staff,2003-09-19,2003.0,9.0,http://www.npr.org/2003/09/19/1436453/musician...,It’s been more than 40 years since David Bowie...
3,708,57718,61037,83402,Nancy Reagan’s Alzheimer’s Crusade,Fox News,NaN,2004-06-06,2004.0,6.0,https://web.archive.org/web/20160306220058/htt...,When Nancy Reagan (search) appeared at a ga...
4,4358,61403,64724,87592,One final thought on,Talking Points Memo,NaN,2004-11-18,2004.0,11.0,https://web.archive.org/web/20170103031234/htt...,Josh Marshall is editor and publisher of Talki...


In [34]:
for col in df:
    print ("column", col, ":", type(df[col][0]))
#Deletes unnecessary columns

#Sets manageable range for working data set
new_df = df[5000:6000]
new_df.head()

column Unnamed: 0 : <class 'numpy.int64'>
column index : <class 'numpy.int64'>
column Unnamed: 0.1 : <class 'numpy.int64'>
column id : <class 'numpy.int64'>
column title : <class 'str'>
column publication : <class 'str'>
column author : <class 'str'>
column date : <class 'str'>
column year : <class 'numpy.float64'>
column month : <class 'numpy.float64'>
column url : <class 'str'>
column content : <class 'str'>


,Unnamed: 0,index,Unnamed: 0.1,id,title,publication,author,date,year,month,url,content
5000,14259,73869,77312,117697,Ex-Spies Say That Clinton’s Illegal Server Tri...,National Review,Deroy Murdock,2016-02-01,2016.0,2.0,http://www.nationalreview.com/article/430597/h...,Three veterans of American intelligence are ho...
5001,14260,73870,77313,117698,The Art of the Deal-Breaker,National Review,Jim Geraghty,2016-02-01,2016.0,2.0,http://www.nationalreview.com/article/430586/d...,"For the past year, one political power player ..."
5002,14261,73871,77314,117699,"Why Do Top Conservatives Pardon Trump, but Con...",National Review,Deroy Murdock,2016-02-01,2016.0,2.0,http://www.nationalreview.com/article/430634/d...,"As Alexander Pope once said, “To err is human,..."
5003,8930,68540,71948,104318,Here’s What Facebook Knows About Young People...,Buzzfeed News,Matthew Zeitlin,2016-02-01,2016.0,2.0,https://web.archive.org/web/20160201031338/htt...,’ There are lots of affluent young people on F...
5004,8920,68530,71938,104304,LIFE AND DEATH INSIDE AN ISIS PRISON,Buzzfeed News,Mike Giglio,2016-02-01,2016.0,2.0,https://web.archive.org/web/20160201031338/htt...,"TEL ABYAD, Syria — The jihadis had scrawle..."


In [66]:
corpus = []
for text in new_df['content']:
    corpus.append(text)

titles = []
for title in new_df["title"]:
    titles.append(str(title))
#labels_df starts at df[5000] so we're good on the matching of labels to content

events = []
for event in labels_df["Event"][:1000]:
    events.append(str(event))

corpus[1]

'For the past year, one political power player after another has tried to negotiate with Donald Trump, and gotten bitten for the effort. Yet, like lemmings, each successive figure stepped up to meet the same fate, convinced despite all available evidence that gravity would give them a better deal. These movers and shakers seemed to think they could control Trump, and attempted to reach a bargain with the man who wrote The Art of the Deal. It did not end well for any of them. Fox News’s Roger Ailes gave Trump every conceivable venue and outlet to make his case, often letting him literally phone in interviews. Last week, Trump turned around and skipped out on the network’s debate because he didn’t like Megyn Kelly’s questions back in August and a snarky press release had offended him. Not only did Trump skip the debate, but he staged a competing event for veterans, stealing viewers away from the candidates who did participate. RELATED: In Face of Controversy, Trump Supporters Stand By Th

In [67]:
import nltk
nltk.download('punkt')
  
def TFIDF(corpus):
    def tokenize_and_stem(text):
        tokens = [word for sent in nltk.sent_tokenize(text) for word in nltk.word_tokenize(sent)]
        filtered_tokens = []
        for token in tokens:
            if re.search('[a-zA-Z]', token):
                filtered_tokens.append(token)
        stems = [stemmer.stem(t) for t in filtered_tokens]
        return stems
    
    def tokenize_only(text):
        tokens = [word.lower() for sent in nltk.sent_tokenize(text) for word in nltk.word_tokenize(sent)]
        filtered_tokens = []
        for token in tokens:
            if re.search('[a-zA-Z]', token):
                filtered_tokens.append(token)
        return filtered_tokens
    
    
    totalvocab_stemmed = []
    totalvocab_tokenized = []
    for i in corpus:
        allwords_stemmed = tokenize_and_stem(i)
        totalvocab_stemmed.extend(allwords_stemmed)
        allwords_tokenized = tokenize_only(i)
        totalvocab_tokenized.extend(allwords_tokenized)
    
    #Let's you search with stemmed word to see original format of word
    vocab_frame = pd.DataFrame({'words': totalvocab_tokenized}, index = totalvocab_stemmed)
    print ('there are ' + str(vocab_frame.shape[0]) + ' items in vocab_frame')
    
    tf = TfidfVectorizer(analyzer='word', use_idf=True, stop_words = "english", tokenizer = tokenize_and_stem,min_df = 0)
    
    global tfidf_matrix
    tfidf_matrix =  tf.fit_transform(corpus)
    print(tfidf_matrix.shape)
    terms = tf.get_feature_names()
    
    
    from sklearn.metrics.pairwise import cosine_similarity
    dist = 1 - cosine_similarity(tfidf_matrix)

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\alvin\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [68]:
# Clustering the document with KNN classifier
from sklearn.neighbors import NearestNeighbors
num_neighbors = 350

X = tfidf_matrix
nbrs = NearestNeighbors(n_neighbors=num_neighbors, algorithm='ball_tree').fit(X)
distances, indices = nbrs.kneighbors(X)

C:\Users\alvin\Anaconda3\lib\site-packages\sklearn\neighbors\base.py:217: UserWarning: cannot use tree with sparse input: using brute force
  warnings.warn("cannot use tree with sparse input: "


In [69]:
indices

array([[  0, 765, 655, ..., 151, 396, 595],
       [  1,   9, 240, ..., 857, 550, 386],
       [  2,  33,  31, ..., 336,  85, 626],
       ...,
       [997, 883, 952, ..., 749, 468, 928],
       [998, 726, 881, ..., 854, 481, 647],
       [999, 883, 952, ..., 685, 241, 250]], dtype=int64)

In [70]:
distances

array([[0.        , 1.08316701, 1.11064715, ..., 1.36933383, 1.36936796,
        1.36943233],
       [0.        , 1.08825591, 1.10236844, ..., 1.35880142, 1.35911026,
        1.35924057],
       [0.        , 1.17705717, 1.20737118, ..., 1.36921261, 1.36921337,
        1.36927525],
       ...,
       [0.        , 0.91279977, 1.0046211 , ..., 1.37171503, 1.37175038,
        1.37181553],
       [0.        , 1.24831777, 1.26783928, ..., 1.37517462, 1.37533333,
        1.37536427],
       [0.        , 0.97846322, 1.01297953, ..., 1.36470582, 1.36485265,
        1.3652368 ]])